In [2]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark import SparkFiles
# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
path = "/content/drive/MyDrive/Colab Notebooks/datasets/amazon_reviews_us_Beauty_v1_00.tsv"
spark.sparkContext.addFile(path)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [8]:
high_votes_df = df.filter(df.total_votes >= "20")
high_votes_df.show()
#df.dtypes

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [9]:
percentage_helpful_votes_df = high_votes_df.filter(high_votes_df.helpful_votes/high_votes_df.total_votes >= "0.50") 
percentage_helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [10]:
paid_vine_df = percentage_helpful_votes_df.filter(percentage_helpful_votes_df.vine == 'Y') 
paid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47770601|R106V6GUNWRSSS|B00YAR86BY|     136411552|Philips Sonicare ...|          Beauty|          5|          857|        889|   Y|                N|Improved build, g...|According to spec...| 2015-08-27|
|         US|   52846213|R37N8KZS48W36A|B00YJJXJ06|     791197070|Maybelline New Yo...|          Beauty|          5|    

In [11]:
unpaid_vine_df = percentage_helpful_votes_df.filter(percentage_helpful_votes_df.vine == 'N') 
unpaid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [16]:
from pyspark.sql.functions import sum,avg,max,count
# total_paid_vine_df = paid_vine_df.agg(count(paid_vine_df.review_id))
# total_paid_vine_df.show()
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

647

In [17]:
from pyspark.sql.functions import sum,avg,max,count
five_star_paid_reviews = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
five_star_paid_reviews

229

In [19]:
from pyspark.sql.functions import percent_rank
percentage_five_star_paid_reviews = (five_star_paid_reviews/total_paid_reviews) * 100
percentage_five_star_paid_reviews

35.394126738794434

In [20]:
from pyspark.sql.functions import sum,avg,max,count
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

74113

In [21]:
from pyspark.sql.functions import sum,avg,max,count
five_star_unpaid_reviews = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
five_star_unpaid_reviews

43217

In [22]:
percentage_five_star_unpaid_reviews = (five_star_unpaid_reviews/total_unpaid_reviews) * 100
percentage_five_star_unpaid_reviews

58.312306882733125